In [1]:
using Revise
using GeometryBasics: Vec3f, Point3f, Cylinder
using LinearAlgebra
using GLMakie
using StaticArrays
using VMRobotControl
using VMRobotControl.Splines: CubicSpline
using DifferentialEquations
using MeshIO
include("../functions.jl")

generate_q_init (generic function with 1 method)

In [76]:
m = Mechanism{Float64}("TestMechanism")

add_frame!(m; id = "base_frame")
add_joint!(m, Rigid(Transform(SVector(0.0, -0.035, 0.32)));  parent = root_frame(m), child = "base_frame", id = "base_joint")
add_frame!(m; id="revo_frame_1")
add_joint!(m, Revolute(SVector(0.0, 0.0, 1.0)); parent = "base_frame", child="revo_frame_1", id="revo_joint_1")
add_frame!(m; id="revo_frame_2")
add_joint!(m, Revolute(SVector(1.0, 0.0, 0.0)); parent="revo_frame_1", child="revo_frame_2", id="revo_joint_2")
add_frame!(m; id="ee frame")
add_joint!(m, Rigid(Transform(SVector(0.0,0.0,0.15))); parent="revo_frame_2", child="ee frame", id="ee_joint")

add_coordinate!(m, FrameOrigin("ee frame"); id="ee position")
add_component!(m, PointMass(1.0, "ee position"); id="ee mass")

# add_coordinate!(m, ConstCoord(SVector(0.0, 0.0, 0.0)); id="base position")
# add_coordinate!(m, CoordDifference("base position", "ee position"); id="ee position diff")
# K_matrix = SMatrix{3, 3}(100., 0., 0., 0., 100., 0., 0., 0., 100.)
# add_component!(m, LinearSpring(K_matrix,"ee position diff"); id="spring")

add_coordinate!(m, JointSubspace("revo_joint_1"); id="revo joint 1")
add_component!(m, LinearDamper(0.1, "revo joint 1"); id="revo joint 1 damper")
add_coordinate!(m, JointSubspace("revo_joint_2"); id="revo joint 2")
add_component!(m, LinearDamper(0.1, "revo joint 2"); id="revo joint 2 damper")

I_mat = @SMatrix [
    0.1  0.    0.  ;
    0.    0.1  0.  ;
    0.    0.    0.1
]

# add_inertia!(m, "revo_frame_1", I_mat; id="revo1 inertia")
# add_inertia!(m, "revo_frame_2", I_mat; id="revo2 inertia")
add_inertia!(m, "ee frame", I_mat; id="ee inertia")

2DOF Mechanism{Float64} "TestMechanism" with 5 frames, 4 joints, 4 coordinates, 4 components

In [77]:
cm = compile(m)
kcache = Observable(new_kinematics_cache(cm))

# Setup the figure
fig1 = Figure(size=(700, 350))
ls1 = LScene(fig1[1, 1]; show_axis=false)

robotsketch!(ls1, kcache; scale=0.3, linewidth=2.5, transparency=true)

display(fig1)

GLMakie.Screen(...)

In [78]:
q = [0.1, 0.1]
q̇ = zero_q̇(cm) 

dcache = new_dynamics_cache(cm)
g = VMRobotControl.DEFAULT_GRAVITY
prob = get_ode_problem(dcache, g, q, q̇, 10.0)
sol = solve(prob, Tsit5(), abstol=1e-3, reltol=1e-3)

display(fig1)
animate_robot_odesolution(fig1, sol, kcache, "robot_animation.mp4")

"robot_animation.mp4"